In [ ]:
import requests
import json
def query_llm_rest_pix(mlist,image=None):
        
        url = 'http://<IP>:9109/v1/chat/completions'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer token'
        }
        data = {
            "model": "mistralai/Pixtral-12B-2409"
        }
        data.update({"messages": mlist})

        response = requests.post(url, headers=headers, data=json.dumps(data))
        # print(response.status_code)
        # print(response)
        if response.status_code == 400:
            return "I'm unsure about the answer. Please provide more context or ask a different question."
        else:
            return response.json()["choices"][0]['message']['content']

# prm = """"""

# response = query_llm_rest_pix([{"role": "user", "content": [{"type": "text", "text": prm}]}])


In [ ]:
import os
import json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter

import requests
import json


import time
from chromadb import HttpClient
from chromadb.utils import embedding_functions

chroma_client = HttpClient(host="localhost", port=9158)


from PyPDF2 import PdfReader
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter

chroma_client = HttpClient(host="localhost", port=9158)


pdf1 = "insurance"
pdf2 = "rivers"
pdf3 = "wildlife"



In [84]:
res = chroma_client.list_collections()
res[0].name


'rivers'

In [ ]:
chroma_client

In [ ]:

def query_llm_rest_pix(mlist,image=None):
        
        url = 'http://IP/v1/chat/completions'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer token'
        }
        data = {
            "model": "mistralai/Pixtral-12B-2409"
        }
        data.update({"messages": mlist})

        response = requests.post(url, headers=headers, data=json.dumps(data))
        # print(response.status_code)
        # print(response)
        if response.status_code == 400:
            return "I'm unsure about the answer. Please provide more context or ask a different question."
        else:
            return response.json()["choices"][0]['message']['content']



def extract_text_from_pdf(pdf_path: str) -> List[str]:
    reader = PdfReader(pdf_path)
    chunks = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    for page in reader.pages:
        text = page.extract_text()
        paragraphs = text.split('\n\n')
        for paragraph in paragraphs:
            if paragraph.strip():
                chunks.extend(splitter.split_text(paragraph.strip()))
    print("chunks--------------------------------------\n",len(chunks))
    return chunks


extract_keywords = False
# Function to load, chunk, and store PDFs into vector stores
def chunk_and_store(pdf_path: str, db_name: str,):
    # Load PDF
    documents = []
    print("pdf:", pdf_path)
    loader = PyPDFLoader(pdf_path)
    documents.extend(loader.load())
    text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)

    chunks = text_splitter.split_documents(documents)
    combined_chunks = " ".join([i.page_content for i in chunks])


    # text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)

    # docs = text_splitter.split_documents(documents)
    collection = chroma_client.get_or_create_collection(db_name,embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2"))

    collection.add(ids=[f"doc_{i}" for i in range(len(chunks))], documents=[i.page_content for i in chunks],metadatas=[i.metadata for i in chunks])

    print(f"Stored {len(chunks)} chunks in vector store.")
    print("combined_chunks--------------------------------------\n",combined_chunks)
    # Extract keywords from chunks using LLM
    # combined_chunks = " ".join(chunks)
    if extract_keywords:
        # print(chunk)
        prompt = f"""Extract only the keywords from the following text. Return response in a python list. Do not include any other text in the response:\n\n
                {combined_chunks}\n\n
                example response:
                ['keyword1', 'keyword2', 'keyword3']
                """
        # print(prompt)
        
        response = query_llm_rest_pix([{"role": "user", "content": [{"type": "text", "text": prompt}]}])
        print(response)
        import re
        import ast   
        match = re.search(r"\[.*\]", response, re.DOTALL)

        if match:
            keyword_list = ast.literal_eval(match.group(0))  # Safely convert to Python list
            print(keyword_list)
        else:
            keyword_list = []
            print("No list found in response.")
        # Save keywords to a dictionary
        keywords_dict = {}
        keywords_dict[db_name] = keyword_list

        # Open the file and update the dictionary
        keywords_file = "/home/vikasnr/codebase/crsl/scratch/new_keywords.json"
        if os.path.exists(keywords_file):
            with open(keywords_file, "r") as f:
                existing_keywords = json.load(f)
        else:
            existing_keywords = {}

        existing_keywords.update(keywords_dict)

        with open(keywords_file, "w") as f:
            json.dump(existing_keywords, f, indent=4)

# # Function to process PDF files and store embeddings
# def process_pdf_files(pdf_file: str, vector_store: Chroma):
#     # Load PDF files

#     loader = PyMuPDFLoader(pdf_file)
#     documents = loader.load()
   
#     # Split text recursively
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     split_docs = text_splitter.split_documents(documents)
   
#     # Store embeddings
#     vector_store.add_documents(split_docs)



# Load PDFs
pdf_files = {pdf1: f"/home/vikasnr/codebase/crsl/pdfs/{pdf1}.pdf", 
             pdf2: f"/home/vikasnr/codebase/crsl/pdfs/{pdf2}.pdf", 
             pdf3: f"/home/vikasnr/codebase/crsl/pdfs/{pdf3}.pdf"}

for db_name, pdf_path in pdf_files.items():
    chunk_and_store(pdf_path, db_name)



pdf: /home/vikasnr/codebase/crsl/pdfs/insurance.pdf
Stored 17 chunks in vector store.
combined_chunks--------------------------------------
 MODULE - 3
Practice of Life Insurance
Notes
 103
Claims and Settlement
DIPLOMA IN INSURANCE SERVICES
7.0 INTRODUCTION
The Insurance Policy is taken by the consumers to compensate
them in the event of happening of an unforeseen event. It is a
hedge against unavoidable circumstances. In general
insurance the loss is payable only on happening of some
specific event. If the insured does not suffer any loss no claim
is paid to him. The premium is charged on yearly basis and
no accumulation takes place. However the scenario is different
in case of life insurance. If the insured dies during the policy
period he gets the sum assured along with the bonus accrued
under the policy if any. If the insured survives the policy period
he gets the maturity amount accrued under the policy. In this
lesson we shall learn the various aspects in settlement of life
insu

In [ ]:
from chromadb.api.client import Client

type(chroma_client)


chromadb.api.client.Client



chromadb.api.client.Client

In [8]:
# List all documents in vector_store_1
collection = chroma_client.get_or_create_collection(pdf2,embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2"))
results = collection.query(query_texts=["What are the function of rivers"], n_results=5, include=["documents"])
results["documents"]
# retrieved_texts = [doc["text"] for doc in results]



/home/vikasnr/miniconda3/envs/mispix_clone/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/vikasnr/miniconda3/envs/mispix_clone/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[["engineering structures such as canals, pipes, or culverts. Rivers make up only a tiny \npercentage of the world's water. Ninety seven percent of the world's water is held in the oceans, of the remaining three percent which is  fresh water, three quarters is stored as \nland ice (i.e., in glaciers, ice caps), lakes c ontain half a percent, soil moisture 0.05%, \nand river channels only 0.025%. Rivers thus represent only one four-thousandth of the \nplanet's total water. Although rivers make up a small percen tage of the landscape, their \nimpact on the landscape extends far beyond the channel itself.   \n \nFrom a geomorphologic perspective, rivers have three main functions; to erode, \ntransport, and deposit sediment. In the proc ess of accomplishing these three tasks, rivers \ndestroy old landforms and create new ones. From a hydrological pers pective, rivers are \nan important component of the hydrological cy cle. Rivers are also  significant in a",
  "%. Rivers thus represent onl

In [9]:

from langchain.chat_models import ChatOpenAI


# No openai free credits :) using the local model

def get_llm():

    return ChatOpenAI(
    model="mistralai/Pixtral-12B-2409",
    temperature=0,
    max_retries=2,
    api_key="...",
    base_url="http://0.0.0.0:9109/v1",
)
get_llm().invoke("Hello, how are you?")

/tmp/ipykernel_1949658/2560089988.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  return ChatOpenAI(


AIMessage(content="Hello! I'm functioning well, thank you. I'm here to help. How about you? How are you doing today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 9, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Pixtral-12B-2409', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c2418962-11cf-438b-9360-6f0cdaaa35fc-0')

In [59]:
import json

# Path to the final.json file
file_path = "final.json"

# Read the JSON file
with open(file_path, "r") as file:
    keywords_data = json.load(file)

keywords_data
# # Print the keywords
# insurance_keys = keywords_data["insurance"]
# rivers_keys = keywords_data["rivers"]
# wildlife_keys = keywords_data["wildlife"]
# print(insurance_keys)
# print(rivers_keys)
# print(wildlife_keys)


{'keywords': {'insurance': ['MODULE',
   '3',
   'Practice',
   'Life',
   'Insurance',
   'Notes',
   'Claims',
   'and',
   'Settlement',
   'DIPLOMA',
   'INSURANCE',
   'SERVICES',
   'INTRODUCTION',
   'OBJECTIVES',
   'CLAIM',
   'SETTLEMENT',
   'Payment',
   'claim',
   'life',
   'insurance',
   'premium',
   'Survival',
   'Benefit',
   'Claim',
   'Maturity',
   'Benfit',
   'Claim',
   'Death',
   'Benefit',
   'Claim',
   'procedure',
   'documents',
   'forms',
   'Insurance',
   'policy',
   'original',
   'policy',
   'document',
   'age',
   'proof',
   'intimation',
   'death',
   'nominee',
   'received',
   'claim',
   'nomination',
   'policy',
   'life',
   'assured',
   'disappeared',
   'premature',
   'death',
   'claim',
   'Indian',
   'Evidence',
   'Act',
   'probate',
   'succession',
   'certificate',
   'administrator',
   'general',
   'death',
   'occurred',
   'claim',
   'concession',
   'ex',
   'gratia',
   'claim',
   'Ex',
   'gratia',
   'claims

In [ ]:
import getpass
import os

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = ""


from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

print(llm.invoke("Hello, how are you?"))


content="Hello! I'm functioning as intended, thank you. How about you? How's your day going?" additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 32, 'completion_tokens': 23}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'} id='run-be1ed9ed-bf28-4612-8396-42b2e426c952-0' usage_metadata={'input_tokens': 9, 'output_tokens': 23, 'total_tokens': 32}


In [ ]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

tools = [get_word_length]


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)


from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)



from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
list(agent_executor.stream({"input": "How many letters in the word eudca"}))


answers = [step for step in agent_executor.stream({"input": "How many letters in the word eudca"})]
print(answers[-1]['output'] )




> Entering new None chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


> Entering new None chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.
The word "eudca" has 5 letters.


In [60]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage,SystemMessage
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType

import json
system  = f"""You are an expert at routing a user question to the appropriate data source(s).

    Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
    ["insurance"] for insurance-related queries
        if the question contains any of the following keywords: {insurance_keys}
    ["river"] for rivers-related queries
        if the question contains any of the following keywords: {rivers_keys}
    ["wildlife"] for wildlife-related queries
        if the question contains any of the following keywords: {wildlife_keys}
    If there are multiple data sources, you should respond.
        ["insurance", "rivers"]

    If the question does not contain any of the keywords, you should not route the question to any data source.
    Respond: 
        ["insurance", "rivers", "wildlife"] 

    Dont mention any other text in response execpt the list of data sources
    """
systemx  = f"""You are an expert at routing a user question to the appropriate data source(s). 
    Make use of keywords_data dictionary to route the question to the appropriate data source(s).

    ["insurance"] for insurance-related queries
    ["river"] for rivers-related queries

    ["wildlife"] for wildlife-related queries
    If there are multiple data sources, you should respond.
        ["insurance", "rivers"]

    If the question does not contain any of the keywords, you should not route the question to any data source.
    Respond: 
        ["insurance", "rivers", "wildlife"] 

    Dont mention any other text in response execpt the list of data sources
    """
@tool
def get_keyword_dictionary(question) -> str:
    """ returns the router name"""
    # Prompt
    


    # prompt = ChatPromptTemplate.from_messages([
    #     ("system", system),
    #     ("human", "{question}"),
    # ])


    # # Define router
    # router = prompt | llm

    # # question = """
    # # What is the process for settlement of maturity claims?
    # # """

    # result = router.invoke({"question": question})
    # agents = result.content
    # print(agents)
    print(keywords_data)
    return keywords_data

# get_keyword_dictionary.invoke("abc")
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

tools = [get_keyword_dictionary]
# agent_executor = create_react_agent(llm, tools)
system_prompt = SystemMessage(content=systemx)
agent = create_tool_calling_agent(llm, tools, system_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

user_message = HumanMessage(content="What is the process for settlement of maturity claims?")

agent_executor.invoke(user_message)


# response = agent_executor.invoke([system_prompt, user_message])

# response["messages"]

AttributeError: 'SystemMessage' object has no attribute 'input_variables'

In [63]:
from langchain_core.messages import HumanMessage

# response = llm.invoke([HumanMessage(content="hi!")])
# response.content


model_with_tools = llm.bind_tools(tools)
user_message = HumanMessage(content="What is the process for settlement of maturity claims?")

response = model_with_tools.invoke([user_message])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages
systemx  = f"""You are an expert at routing a user question to the appropriate data source(s). 
    Make use of keywords_data dictionary to route the question to the appropriate data source(s).

    ["insurance"] for insurance-related queries
    ["river"] for rivers-related queries

    ["wildlife"] for wildlife-related queries
    If there are multiple data sources, you should respond.
        ["insurance", "rivers"]

    If the question does not contain any of the keywords, you should not route the question to any data source.
    Respond: 
        ["insurance", "rivers", "wildlife"] 

    Dont mention any other text in response execpt the list of data sources
    """
# Define the chat prompt template
prompt = ChatPromptTemplate(
    input_variables=["agent_scratchpad", "input"],
    optional_variables=["chat_history"],
    messages=[
        SystemMessage(content=systemx),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessage(content="{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

res = agent_executor.invoke({"input": "What is the process for settlement of maturity claims?"})

ContentString: 
ToolCalls: [{'name': 'get_keyword_dictionary', 'args': {'question': 'settlement of maturity claims'}, 'id': 'LE6GO0YcA', 'type': 'tool_call'}]
{'keywords': {'insurance': ['MODULE', '3', 'Practice', 'Life', 'Insurance', 'Notes', 'Claims', 'and', 'Settlement', 'DIPLOMA', 'INSURANCE', 'SERVICES', 'INTRODUCTION', 'OBJECTIVES', 'CLAIM', 'SETTLEMENT', 'Payment', 'claim', 'life', 'insurance', 'premium', 'Survival', 'Benefit', 'Claim', 'Maturity', 'Benfit', 'Claim', 'Death', 'Benefit', 'Claim', 'procedure', 'documents', 'forms', 'Insurance', 'policy', 'original', 'policy', 'document', 'age', 'proof', 'intimation', 'death', 'nominee', 'received', 'claim', 'nomination', 'policy', 'life', 'assured', 'disappeared', 'premature', 'death', 'claim', 'Indian', 'Evidence', 'Act', 'probate', 'succession', 'certificate', 'administrator', 'general', 'death', 'occurred', 'claim', 'concession', 'ex', 'gratia', 'claim', 'Ex', 'gratia', 'claims', 'options', 'IRDA', 'REGULATION', 'POLICYHOLDERS'

In [75]:
llm

ChatMistralAI(client=<httpx.Client object at 0x7f64e69b73d0>, async_client=<httpx.AsyncClient object at 0x7f64e4c85b40>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', max_retries=2, model='mistral-large-latest', temperature=0.0)

In [78]:
retrieved_texts = "Some relevant information from the knowledge base."
query = "What is the impact of AI on healthcare?"
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage

# prompt = ChatPromptTemplate(

#     messages=[
#         SystemMessage(content="You are a helpful assistant."),
#         MessagesPlaceholder(variable_name="chat_history", optional=True),
#         HumanMessage(content="Answer based on the following: {retrieved_texts}\nQuestion: {input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )

messages = [
    (
        "system",
        "You are a helpful assistant."
    ),
    ("human", f"Answer based on the following: {retrieved_texts}\nQuestion: {input}"),
]

ai_msg = llm.invoke(messages)
ai_msg.content

"It seems like you've provided a Python method reference, specifically the `raw_input` method of an `IPythonKernel` object, rather than a clear question or topic to discuss. To assist you better, could you please provide more context or clarify your question? Here are a few examples of how I can help:\n\n1. **Answering questions** based on the information provided.\n2. **Explaining concepts** related to the Python programming language, Jupyter notebooks, or IPython kernel.\n3. **Providing code snippets** or examples to illustrate certain functionality.\n\nPlease provide more details or ask a specific question, and I'll do my best to assist you."

In [71]:
import ast


data_sources = ast.literal_eval(res['output'])
print(data_sources)

['insurance', 'rivers', 'wildlife']


In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage

systemx  = f"""You are an expert at routing a user question to the appropriate data source(s).

    Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
    ["insurance"] for insurance-related queries
    ["river"] for rivers-related queries

    ["wildlife"] for wildlife-related queries
    If there are multiple data sources, you should respond.
        ["insurance", "rivers"]

    If the question does not contain any of the keywords, you should not route the question to any data source.
    Respond: 
        ["insurance", "rivers", "wildlife"] 

    Dont mention any other text in response execpt the list of data sources
    """
# Define the chat prompt template
prompt = ChatPromptTemplate(
    input_variables=["agent_scratchpad", "input"],
    optional_variables=["chat_history"],
    messages=[
        SystemMessage(content=systemx),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessage(content="{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


System: You are an expert at routing a user question to the appropriate data source(s).

    Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
    ["insurance"] for insurance-related queries
    ["river"] for rivers-related queries

    ["wildlife"] for wildlife-related queries
    If there are multiple data sources, you should respond.
        ["insurance", "rivers"]

    If the question does not contain any of the keywords, you should not route the question to any data source.
    Respond: 
        ["insurance", "rivers", "wildlife"] 

    Dont mention any other text in response execpt the list of data sources
    
Human: {input}


In [20]:
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType

import json
system  = f"""You are an expert at routing a user question to the appropriate data source(s).

    Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
    ["insurance"] for insurance-related queries
        if the question contains any of the following keywords: {insurance_keys}
    ["river"] for rivers-related queries
        if the question contains any of the following keywords: {rivers_keys}
    ["wildlife"] for wildlife-related queries
        if the question contains any of the following keywords: {wildlife_keys}
    If there are multiple data sources, you should respond.
        ["insurance", "rivers"]

    If the question does not contain any of the keywords, you should not route the question to any data source.
    Respond: 
        ["insurance", "rivers", "wildlife"] 

    Dont mention any other text in response execpt the list of data sources
    """

@tool
def get_keyword_dictionary(question) -> str:
    """ returns the router name"""
    # Prompt
    


    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "{question}"),
    ])


    # Define router
    router = prompt | llm

    # question = """
    # What is the process for settlement of maturity claims?
    # """

    result = router.invoke({"question": question})
    agents = result.content
    print(agents)
    return agents

# get_keyword_dictionary.invoke("abc")

tools = [get_keyword_dictionary]



from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Only return the response from tool",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at routing user questions to the appropriate data source(s)."),
    ("user", "{input}"),
])


# Initialize Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Ensures only tool outputs are returned
    verbose=True
)

# Invoke the agent
response = agent.invoke({"input": "What is the process for settlement of maturity claims?"})
print(response)

# llm_with_tools = llm.bind_tools(tools)


# from langchain.agents.format_scratchpad.openai_tools import (
#     format_to_openai_tool_messages,
# )
# from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_to_openai_tool_messages(
#             x["intermediate_steps"]
#         ),
#     }
#     | prompt
#     | llm_with_tools
#     | OpenAIToolsAgentOutputParser()
# )



# from langchain.agents import AgentExecutor

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# # list(agent_executor.stream({"input": "What is the process for settlement of maturity claims?"}))


# answers = [step for step in agent_executor.stream({"input": "What is the process for settlement of maturity claims?"})]
# print(answers[-1]['output'] )


/tmp/ipykernel_2156331/2454597824.py:74: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...


HTTPStatusError: Error response 422 while fetching https://api.mistral.ai/v1/chat/completions: {"object":"error","message":{"detail":[{"type":"extra_forbidden","loc":["body","functions"],"msg":"Extra inputs are not permitted","input":[{"name":"get_keyword_dictionary","description":"returns the router name","parameters":{"properties":{"question":{}},"required":["question"],"type":"object"}}],"url":"https://errors.pydantic.dev/2.10/v/extra_forbidden"}]},"type":"invalid_request_error","param":null,"code":null}

In [ ]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

tools = [get_word_length]


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


from langchain_core.prompts import ChatPromptTemplate


# Prompt
system = f"""You are an expert at routing a user question to the appropriate data source(s).

Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
["insurance"] for insurance-related queries
    if the question contains any of the following keywords: {insurance_keys}
["river"] for rivers-related queries
    if the question contains any of the following keywords: {rivers_keys}
["wildlife"] for wildlife-related queries
    if the question contains any of the following keywords: {wildlife_keys}
If there are multiple data sources, you should respond.
    ["insurance", "rivers"]

If the question does not contain any of the keywords, you should not route the question to any data source.
Respond: 
    ["insurance", "rivers", "wildlife"] 

Dont mention any other text in response except the list of data sources
"""


prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}"),
])
llm_with_tools = llm.bind_tools(tools)


from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)



from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
question = """
What is the process for settlement of maturity claims?
"""
list(agent_executor.stream({"input": "What is the process for settlement of maturity claims?"}))


answers = [step["output"]["content"] for step in agent_executor.stream({"input": "How many letters in the word eudca"})]
print(answers)





> Entering new None chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'question'}.  Expected: ['question'] Received: ['input', 'agent_scratchpad']\nNote: if you intended {question} to be part of the string and not a variable, please escape it with double curly braces like: '{{question}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:
tools = [get_word_length]


llm_with_tools = llm.bind_tools(tools)



In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:

# Prompt
system = f"""You are an expert at routing a user question to the appropriate data source(s).

Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
["insurance"] for insurance-related queries
    if the question contains any of the following keywords: {insurance_keys}
["river"] for rivers-related queries
    if the question contains any of the following keywords: {rivers_keys}
["wildlife"] for wildlife-related queries
    if the question contains any of the following keywords: {wildlife_keys}
If there are multiple data sources, you should respond.
    ["insurance", "rivers"]

If the question does not contain any of the keywords, you should not route the question to any data source.
Respond: 
    ["insurance", "rivers", "wildlife"] 

Dont mention any other text in response execpt the list of data sources
"""


prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}"),
])

llm = get_llm()

# Define router
router = prompt | llm

question = """
What is the process for settlement of maturity claims?
"""

result = router.invoke({"question": question})
print(result)

content='["insurance"]' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 1000, 'total_tokens': 1005, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Pixtral-12B-2409', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-10b2ca93-211d-410c-bfbe-c4d5d7d85282-0'


In [ ]:
agents = result.content
print(agents)
agent_list = json.loads(agents)
print(agent_list)


["insurance"]
['insurance']


list

In [73]:

def pdf_agent(query: str, db_name: str):
    """Handles retrieval from ChromaDB and passes relevant chunks to LLM."""
    collection = chroma_client.get_or_create_collection(db_name,embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2"))

    results = collection.query(query_texts=[query], n_results=5, include=["documents"])   
    
    return results['documents']

retrieved_texts = []
for agent in agent_list:
    retrieved_texts.append(pdf_agent(question, agent))


In [76]:
def aggregator_agent(retrieved_texts: List[str], query: str):
    """Aggregates the retrieved texts and passes them to LLM."""

    print(retrieved_texts)
    response = query_llm_rest_pix([
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Answer based on the following :        {retrieved_texts}\n        Question: {query}"}
    ])

    return response

aggregator_agent(retrieved_texts,question)



[[['If the maturity claim is demanded within one year, before the\nmaturity it is called a discounted maturity claim. This amountis much less than the maturity claim.\n7.2.3 Death ClaimIf the life assured dies during the term of the policy, the death\nclaim arises. If the death has taken place within the first twoyears of the commencement of the policy, it is called an earlydeath claim and if the death has taken after 2 years, it iscalled a non early death claim.\nINTEXT QUESTIONS 7.1\n1. When the maturity claim is payable?\n2. When the death claim is payable?\n7.3 CLAIM DOCUMENTS & FORMS AND SETTLEMENT\nPROCEDURE\nWe will discuss in this section the insurance documents\nnecessary at the time of the final payment. The final paymentmay relate to the maturity or death claim payment.\n7.3.1 The documents required for payment of maturity\nclaim :\n(i) Age proof, if age is not admitted.\n(ii)Original policy document for cancellation.', '2. What is succession certificate?\n7.4. PROCEDURE OF 

'The process for the settlement of maturity claims in life insurance involves several steps:\n\n1. **Policy Maturity**: The policyholder must live through the duration of the policy to become eligible for the maturity value.\n\n2. **Notification**: The insurer sends a claim discharge voucher to the policyholder well before the date of maturity. This voucher needs to be signed and witnessed.\n\n3. **Required Documents**: To receive the maturity value, the policyholder must submit:\n   - Age proof, if age was not admitted at the time of proposal.\n   - The original policy document, which will be canceled upon payment.\n\n4. **Verification**: If age was not admitted during the proposal stage, age proof must be submitted before the payment of the maturity value.\n\n5. **Payment**: Once all the necessary documents are submitted and verified, the insurer processes the payment of the maturity claim.\n\n6. **No Nomination Role**: Since the policyholder is alive, the nomination (if any) does no